In [1]:
import sys, os
sys.path.insert(0, '..')

In [2]:
import torch
from models.weighted_node2vec import WeightedNode2Vec
from torch_geometric.nn.models import Node2Vec
from dataset import triplet_dataset
from utils.config import *
from tqdm import tqdm, trange
from dataset.pokec_data import PokecDataFrame
import gc
from utils.link_prediction import *
import residual2vec as rv

import warnings
warnings.filterwarnings("ignore")
gc.enable()

In [3]:

window_length = 5
num_walks = 10
dim = 128
walk_length = 80
NUM_WORKERS = 4


In [4]:
d = PokecDataFrame()
edge_index, num_nodes = d.edge_index, d.X.shape[0]

Using existing file soc-pokec-profiles.txt.gz
Using existing file soc-pokec-relationships.txt.gz


In [5]:
node_to_vec = WeightedNode2Vec(num_nodes=num_nodes,
                         group_membership=d.get_grouped_col(),
                         weighted_adj="../data/pokec_crosswalk_adj.npz",
                         edge_index=edge_index, 
                         embedding_dim=16, walk_length=walk_length, 
                         context_size=2, ).to(DEVICE)

In [6]:
del d
gc.collect()

5258

In [7]:
k = "degree-unbiased-gcn"
model = rv.residual2vec_sgd(
    noise_sampler=rv.ConfigModelNodeSampler(),
    window_length=window_length,
    num_walks=num_walks,
    walk_length=walk_length
).fit()

In [8]:
d = triplet_dataset.TripletPokecDataset(node2vec=node_to_vec)
dataloader = triplet_dataset.NeighborEdgeSampler(d, batch_size=model.batch_size, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

  0%|                                                                                                                                   | 0/1 [00:00<?, ?it/s]

training node2vec


Epoch 001, Loss: 0.0001: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [03:05<00:00, 185.83s/it]


In [11]:
m = GCNLinkPrediction(in_channels=d.num_features, embedding_size=128, hidden_channels=64, num_layers=5, num_embeddings=d.num_embeddings)
# model.transform(model=m, dataloader=dataloader)

In [10]:
d.num_features

16

In [12]:
torch.save(m.state_dict(), k)